**Change runtime type --> GPU**

**Mount drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import tensorflow as tf
import numpy as np
from scipy.special import softmax
import time
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Set only the first GPU as visible
        tf.config.set_visible_devices(gpus[0], 'GPU')
        # Allow memory growth to allocate memory dynamically on the GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU configuration successful.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected.")

GPU configuration successful.


In [ ]:
file_path = "drive/MyDrive/tinyML_projects/KWS/left.wav"             #  left.wav
SAMPLES_TO_CONSIDER = 16000
sr = 16000
mapping = [ 'background_noise','unknown',  'down']

frame_length = int(sr * ( 32 / 1000))
hop_length = int(sr * ( 16 / 1000))

# kearas model

In [ ]:
#SAVED_MODEL_PATH = "/home/soumen/Soumen/model/KWS/project_1/2D_CNN.keras"
#SAVED_MODEL_PATH = "/home/soumen/Soumen/model/KWS/project_1/1D_CNN.keras"
SAVED_MODEL_PATH = "drive/MyDrive/tinyML_projects/1D_CNN.keras"

In [ ]:
signal, sample_rate = librosa.load(file_path)
#print(sample_rate)
if len(signal)>=SAMPLES_TO_CONSIDER:
    signal=signal[0:SAMPLES_TO_CONSIDER]
else:
    npad = SAMPLES_TO_CONSIDER - len(signal)
    signal=np.pad(signal, pad_width=npad, mode='constant', constant_values=0)[npad:]

signal=signal.astype(np.float32)
f=librosa.feature.mfcc(y=signal, sr=sr, win_length = frame_length,hop_length=hop_length,n_mfcc=13, n_fft = frame_length,center=0)

f=f.T
###### For 1-D CNN #######
f = f.reshape(1,f.shape[0],f.shape[1])
#print(f.shape)
##########################
###### For 2-D CNN #######
# f = f.reshape(1,f.shape[0],f.shape[1],1)
# print(f.shape)
##########################
#load the model
model = tf.keras.models.load_model(SAVED_MODEL_PATH)

start_time = time.time()
predictions = model.predict(f)
end_time = time.time()

# Calculate the time taken for the prediction
prediction_time = end_time - start_time

#Posterior Handling
np.set_printoptions(precision=3)
scores = np.ravel(predictions)
output=softmax(scores)
print(output)
predicted_index = np.argmax(output, axis=0)
print(mapping[predicted_index])

print(f"Time taken for prediction: {prediction_time} seconds")

# Get the size of the model in bytes
model_size_bytes = os.path.getsize(SAVED_MODEL_PATH)

# Convert to kilobytes (KB) and megabytes (MB)
model_size_kb = model_size_bytes / 1024
model_size_mb = model_size_kb / 1024

print(f"Model size: {model_size_bytes} bytes")
print(f"Model size: {model_size_kb:.2f} KB")
print(f"Model size: {model_size_mb:.2f} MB")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
[0.212 0.576 0.212]
unknown
Time taken for prediction: 0.2586708068847656 seconds
Model size: 44295470 bytes
Model size: 43257.29 KB
Model size: 42.24 MB


In [ ]:
from tensorflow import lite
from tensorflow.keras import models

In [ ]:
#############  2D_CNN  #############
#keras_model_filename = '/home/soumen/Soumen/model/KWS/project_1/2D_CNN.keras'
#tflite_filename = '/home/soumen/Soumen/model/KWS/project_1/2D_CNN.tflite'

#############  1D_CNN  #############
keras_model_filename = "drive/MyDrive/tinyML_projects/1D_CNN.keras"
tflite_filename = 'drive/MyDrive/tinyML_projects/1D_CNN.tflite'

#############  TCN  #############
#keras_model_filename = '/home/soumen/Soumen/model/KWS/project_1/TCN.keras'
#flite_filename = '/home/soumen/Soumen/model/KWS/project_1/TCN.tflite'

In [ ]:
# Convert model to TF Lite model
model = models.load_model(keras_model_filename)
converter = lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open(tflite_filename, 'wb').write(tflite_model)

Saved artifact at '/tmp/tmpk976137q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 61, 13), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  135466570835648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466570515728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543288336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543295552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543300304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543582720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543587472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543593104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543593632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543594160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135466543761712: T

14759156

# TFLite model

In [ ]:
#SAVED_MODEL_PATH = "/home/soumen/Soumen/model/KWS/project_1/2D_CNN.tflite"
SAVED_MODEL_PATH = "drive/MyDrive/tinyML_projects/1D_CNN.tflite"
#SAVED_MODEL_PATH = "/home/soumen/Soumen/model/KWS/project_1/TCN.tflite"

In [ ]:
# Load the audio file
signal, sample_rate = librosa.load(file_path, sr=None)  # Use sr=None to preserve the original sampling rate

# Ensure the signal length matches SAMPLES_TO_CONSIDER
if len(signal) >= SAMPLES_TO_CONSIDER:
    signal = signal[0:SAMPLES_TO_CONSIDER]
else:
    npad = SAMPLES_TO_CONSIDER - len(signal)
    signal = np.pad(signal, (0, npad), mode='constant', constant_values=0)

# Convert signal to float32
signal = signal.astype(np.float32)

# Extract MFCC features
f = librosa.feature.mfcc(y=signal, sr=sample_rate, win_length=frame_length,
                         hop_length=hop_length, n_mfcc=13, n_fft=frame_length, center=False)

# Transpose the MFCC matrix
f = f.T

# Reshape for CNN input
###### For 1-D CNN #######
f = f.reshape(1, f.shape[0], f.shape[1])

###### For 2-D CNN #######
# f = f.reshape(1, f.shape[0], f.shape[1], 1)

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=SAVED_MODEL_PATH)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], f)

# Measure the time taken for prediction
start_time = time.time()
interpreter.invoke()
end_time = time.time()

# Get the output tensor
predictions = interpreter.get_tensor(output_details[0]['index'])

# Calculate the time taken for the prediction
prediction_time = end_time - start_time

# Handle the output predictions
np.set_printoptions(precision=3)
scores = np.ravel(predictions)
output = softmax(scores)
print(output)

# Determine the predicted class
predicted_index = np.argmax(output, axis=0)
print(mapping[predicted_index])

print(f"Time taken for prediction: {prediction_time} seconds")


# Get the size of the model in bytes
model_size_bytes = os.path.getsize(SAVED_MODEL_PATH)

# Convert to kilobytes (KB) and megabytes (MB)
model_size_kb = model_size_bytes / 1024
model_size_mb = model_size_kb / 1024

print(f"Model size: {model_size_bytes} bytes")
print(f"Model size: {model_size_kb:.2f} KB")
print(f"Model size: {model_size_mb:.2f} MB")

[0.212 0.576 0.212]
background_noise
Time taken for prediction: 0.003979206085205078 seconds
Model size: 14759156 bytes
Model size: 14413.24 KB
Model size: 14.08 MB
